### Benchmarking Ben-Or and Tiwari algorithm 

#### Part I - Univariate polynomials over finite fields


In [1]:
# include the ExactSparseInterpolations package from the local dir 
include("../src/ExactSparseInterpolations.jl");


Welcome to Nemo version 0.32.6

Nemo comes with absolutely no warranty whatsoever



In [2]:
using Nemo
using BenchmarkTools
using Primes
using DataFrames

We benchmark on a sequence of polynomials $f_i$ for $i = 1,2,\ldots,$.

- the number of terms in polynomial $f_i$ is $2^i$;
- the degree of all $f_i$ is constant and equals $2^{16}$;

In the first experiment, the ground field is $Z/Z_{3*2^{30} + 1}$. In the second one the field is $F_{5^{12}}$.


In [3]:
# from nanoseconds to seconds
sround(t) = round(t, digits=4);
ns2s(t) = sround(t / 1e9);

In [4]:
function benchmark_benortiwari(R, ns)
    times = (
        total=Float64[],
        padé=Float64[],
        roots=Float64[],
        dlog=Float64[],
        vandermonde=Float64[]
    )
    K = base_ring(R)
    x = first(gens(R))
    randpoly(term_range, exp_range) = rand(R, term_range, exp_range)
    D = 2^16
    for i in ns
        n = 2^i
        f = mod(x^D + randpoly(n-1:n-1, 0:D), x^(D + 1) - 1)
        @assert (length(f) - n) / n < 0.01
        @assert degree(f, 1) == D
        blackbox = ExactSparseInterpolations.Blackbox(f)
        bot = ExactSparseInterpolations.FasterBenOrTiwari(R, length(f))
        @assert f == ExactSparseInterpolations.interpolate!(bot, blackbox)
        bench = @benchmarkable ExactSparseInterpolations.interpolate!($bot, $blackbox) samples=3
        result = minimum(run(bench)).time
        push!(times.total, ns2s(result))
        internaltimes = ExactSparseInterpolations._runtime_benortiwari_dump()
        push!(times.padé, sround(internaltimes.padé))
        push!(times.roots, sround(internaltimes.roots))
        push!(times.dlog, sround(internaltimes.dlog))
        push!(times.vandermonde, sround(internaltimes.vandermonde))
    end
    times
end

benchmark_benortiwari (generic function with 1 method)

In [5]:
Zp = GF(2^30*3 + 1)
R_zp, (x_zp,) = PolynomialRing(Zp, ["x"])
range_zp = 7:13
times_zp = benchmark_benortiwari(R_zp, range_zp);

In [6]:
# K = F_5^12
Fqn = FqNmodFiniteField(fmpz(5), 12, :z)
R_fqn, (x_fqn,) = PolynomialRing(Fqn, ["x"])
range_fqn = 5:11
times_fqn = benchmark_benortiwari(R_fqn, range_fqn);

In [9]:
df_zp = DataFrame(
    nterms=["2^$i" for i in range_zp], 
    total=times_zp.total,
    padé=times_zp.padé,
    roots=times_zp.roots,
    dlog=times_zp.dlog,
    vandermonde=times_zp.vandermonde
)
df_zp

Row,nterms,total,padé,roots,dlog,vandermonde
,String,Float64,Float64,Float64,Float64,Float64
1,2^7,0.0145,0.001,0.009,0.0019,0.0003
2,2^8,0.0455,0.0022,0.0315,0.004,0.0007
3,2^9,0.1558,0.0048,0.1113,0.0081,0.0019
4,2^10,0.6158,0.0126,0.4651,0.0168,0.0048
5,2^11,2.3376,0.0284,1.796,0.0335,0.0131
6,2^12,7.3521,0.0656,5.4512,0.0644,0.035
7,2^13,24.1499,0.1598,17.22,0.1303,0.0881


In [10]:
df_fqn = DataFrame(
    nterms=["2^$i" for i in range_fqn], 
    total=times_fqn.total,
    padé=times_fqn.padé,
    roots=times_fqn.roots,
    dlog=times_fqn.dlog,
    vandermonde=times_fqn.vandermonde
)
df_fqn

Row,nterms,total,padé,roots,dlog,vandermonde
,String,Float64,Float64,Float64,Float64,Float64
1,2^5,0.0967,0.004,0.0629,0.0182,0.002
2,2^6,0.3226,0.0093,0.1778,0.0356,0.0062
3,2^7,0.8008,0.0272,0.5062,0.076,0.018
4,2^8,2.099,0.0606,1.296,0.1463,0.0441
5,2^9,5.2864,0.1468,2.1439,0.2869,0.1088
6,2^10,15.65,0.3358,5.7162,0.5786,0.2579
7,2^11,53.5,0.778,16.63,1.4316,0.6301
